# Common functions

By default, **your databases** will be stored in Naas in the **"Inputs"** folder of each tool.<br>
You can edit it by changing the path of the outputs below if you want to connect to any kind of other tools.

*LinkedIn:*
- Profile posts stats
- Company posts stats
- Profile connections
- Company followers
- Profile posts engagements
- Company posts engagements

*YouTube:*
- Video stats

By default, all **your assets** will be store in Naas in the **"Outputs"**:
- Image
- HTML
- Post process data

## Input

### Import libraries

In [ ]:
import naas
import pandas as pd
from naas_drivers import notion, linkedin, youtube
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
from os import path, environ, makedirs
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from naas_drivers.tools.notion import Link, BlockEmbed
import plotly.express as px
from dateutil import tz

### Get data

In [ ]:
def get_data(file_path, storage="Naas"):
    try:
        # Get CSV in Naas
        if storage == "Naas":
            if not naas.is_production():
                try:
                    input_path = path.join(PROD_DIR, file_path)
                    df = pd.read_csv(input_path) 
                except:
                    input_path = path.join(DEV_DIR, file_path)
                    df = pd.read_csv(input_path)
                    return df
            else:
                input_path = path.join(PROD_DIR, file_path)
                df = pd.read_csv(input_path) 
        # Get AWS 
        elif storage == "AWS":
            df = wr.s3.read_parquet(file_path, dataset=True)
    except Exception as e:
        print(e)
        # Empty dataframe returned
        return pd.DataFrame()
    return df

### Setup Variables

In [ ]:
# Timezone
TIMEZONE = naas.secret.get("TIMEZONE")

# Month Rolling
MONTH_ROLLING = 12

# MTD
PERIOD_MTD = "%Y-%m"
ORDER_MTD = "%Y%m"
DISPLAY_MTD = "%b %Y"
TEXT_MTD = "This month"
TEXT2_MTD = "months"
SCENARIO_MTD = "MTD"
CURRENT_MTD = datetime.now().strftime(PERIOD_MTD)

# Datetime format
DATE_FORMAT = "%Y-%m-%d"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

# Naas directory
PROJECT = naas.secret.get("GIT_REPO_NAME")
PROJECT_PATH = naas.secret.get("GIT_REPO_PATH")

# Manage DIR
PROD_DIR = path.join("/", "home", "ftp", "⚡ → Production", PROJECT)
DEV_DIR = PROJECT_PATH
if naas.is_production():
    DIR = PROD_DIR
else:
    DIR = DEV_DIR
    
# Naas Assets file
ASSETS = "Naas_Assets.csv"

### Setup Notion

In [ ]:
NOTION_CONTENT_DB = naas.secret.get("NOTION_CONTENT_DB")
NOTION_TOKEN = naas.secret.get("NOTION_TOKEN")

### Setup LinkedIn

In [ ]:
# LinkedIn cookies
LI_AT = naas.secret.get("LI_AT") # EXAMPLE : "AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2"
JSESSIONID = naas.secret.get("JSESSIONID") # EXAMPLE : "ajax:8379907400220387585"
LINKEDIN_PROFILE_URL = naas.secret.get("LINKEDIN_PROFILE_URL") # EXAMPLE : "ajax:8379907400220387585"
LINKEDIN_COMPANY_URL = naas.secret.get("LINKEDIN_COMPANY_URL") # EXAMPLE : "ajax:8379907400220387585"

# Outputs folders
LINKEDIN_INPUTS = path.join("LinkedIn", "Inputs")
if not path.exists(path.join(DIR, LINKEDIN_INPUTS)):
    makedirs(path.join(DIR, LINKEDIN_INPUTS))

LINKEDIN_OUTPUTS = path.join("LinkedIn", "Outputs")
if not path.exists(path.join(DIR, LINKEDIN_OUTPUTS)):
    makedirs(path.join(DIR, LINKEDIN_OUTPUTS))

# Get LinkedIn ID
def get_identity(file_path, identity):
    df = pd.DataFrame()
    file_path = path.join(DIR, file_path)
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError as e:
        # Empty dataframe returned
        if identity == "profile":
            if naas.secret.get("LINKEDIN_PROFILE_URL"):
                df = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(LINKEDIN_PROFILE_URL)
        elif identity == "company":
            if naas.secret.get("LINKEDIN_COMPANY_URL"):
                df = linkedin.connect(LI_AT, JSESSIONID).company.get_info(LINKEDIN_COMPANY_URL)
        if len(df) > 0:
            df.to_csv(file_path, index=False)
            naas.dependency.add(file_path)
            return df
    return df

# LinkedIn Profile info
LK_PROFILE_IDENTITY = path.join(LINKEDIN_INPUTS, "LINKEDIN_PROFILE.csv")
df_profile = get_identity(file_path=LK_PROFILE_IDENTITY, identity="profile")
LK_PROFILE_ID = ""
LK_FULLNAME = ""
if len(df_profile) > 0:
    LK_PROFILE_ID = df_profile.loc[0, "PROFILE_ID"]
    LK_FIRSTNAME = df_profile.loc[0, "FIRSTNAME"]
    LK_LASTNAME = df_profile.loc[0, "LASTNAME"]
    LK_FULLNAME = f"{LK_FIRSTNAME} {LK_LASTNAME}"
    LK_PROFILE_CHECK = True

# LinkedIn Company info
LK_COMPANY_INFO = path.join(LINKEDIN_INPUTS, "LINKEDIN_COMPANY.csv")
df_company = get_identity(file_path=LK_COMPANY_INFO, identity="company")
LK_COMPANY_ID = ""
LK_COMPANY_NAME = ""
if len(df_company) > 0:
    LK_COMPANY_ID = df_company.loc[0, "COMPANY_ID"]
    LK_COMPANY_NAME = df_company.loc[0, "COMPANY_NAME"]
    LK_COMPANY_CHECK = True
    
# Posts update
LINKEDIN_POSTS_UPDATE = 7
LINKEDIN_MIN_UPDATED_TIME = 300
NO_TOP_FANS = 20

# Linkedin Stype
LINKEDIN = "LinkedIn"
LINKEDIN_LOGO = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/LinkedIn_logo_initials.png/800px-LinkedIn_logo_initials.png"
LINKEDIN_COLOR = "#1293d2"
LINKEDIN_COLOR2 = "#cfe9f6"

# LinkedIn Profile DATABASES
LK_PROFILE_POSTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_POSTS_{LK_PROFILE_ID}.csv')
LK_PROFILE_CONNECTIONS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_CONNECTIONS_{LK_PROFILE_ID}.csv')
LK_PROFILE_POSTS_LIKES = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_POSTS_LIKES_{LK_PROFILE_ID}.csv')
LK_PROFILE_POSTS_COMMENTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_POSTS_COMMENTS_{LK_PROFILE_ID}.csv')
LK_PROFILE_ENGAGEMENTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_ENGAGEMENTS_{LK_PROFILE_ID}.csv')

# LinkedIn Company DATABASES
LK_COMPANY_POSTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_POSTS_{LK_COMPANY_ID}.csv')
LK_COMPANY_FOLLOWERS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_FOLLOWERS_{LK_COMPANY_ID}.csv')
LK_COMPANY_POSTS_LIKES = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_POSTS_LIKES_{LK_COMPANY_ID}.csv')
LK_COMPANY_POSTS_COMMENTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_POSTS_COMMENTS_{LK_COMPANY_ID}.csv')
LK_COMPANY_ENGAGEMENTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_ENGAGEMENTS_{LK_COMPANY_ID}.csv')

### Setup YouTube

In [ ]:
# YouTube credentials
YOUTUBE_API_KEY = naas.secret.get('YOUTUBE_API_KEY')
YOUTUBE_CHANNEL_URL = naas.secret.get('YOUTUBE_CHANNEL_URL')

# Outputs folders
YOUTUBE_INPUTS = path.join("YouTube", "Inputs")
if not path.exists(path.join(DIR, YOUTUBE_INPUTS)):
    makedirs(path.join(DIR, YOUTUBE_INPUTS))

YOUTUBE_OUTPUTS = path.join("YouTube", "Outputs")
if not path.exists(path.join(DIR, YOUTUBE_OUTPUTS)):
    makedirs(path.join(DIR, YOUTUBE_OUTPUTS))

# YouTube Channel info
YT_CHANNEL_NAME = naas.secret.get('YOUTUBE_CHANNEL_NAME')
YT_CHANNEL_ID = YOUTUBE_CHANNEL_URL.split("/channel/")[-1].split("/")[0]

# YouTube
YOUTUBE = "YouTube"
YOUTUBE_LOGO = "https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/YouTube_social_white_square_%282017%29.svg/300px-YouTube_social_white_square_%282017%29.svg.png"
YOUTUBE_COLOR = "#FF0000"

# YouTube videos
YT_VIDEOS = path.join(YOUTUBE_INPUTS, f'YOUTUBE_VIDEOS_{YT_CHANNEL_ID}.csv')

### Setup Twitter

In [ ]:
# Outputs folders
TWITTER_INPUTS = path.join("Twitter", "Inputs")
if not path.exists(path.join(DIR, TWITTER_INPUTS)):
    makedirs(path.join(DIR, TWITTER_INPUTS))

TWITTER_OUTPUTS = path.join("Twitter", "Outputs")
if not path.exists(path.join(DIR, TWITTER_OUTPUTS)):
    makedirs(path.join(DIR, TWITTER_OUTPUTS))

# Twitter
TWITTER = "Twitter"
TWITTER_LOGO = "https://sapiens-uspc.com/wp-content/uploads/2017/10/twitter-logo-vector.png"
TWITTER_COLOR = "#1DA1F2"

### Get file path

In [ ]:
def get_file_path(file_path):
    if naas.is_production():
        file_path = path.join(PROD_DIR, file_path)
    else:
        file_path = path.join(DEV_DIR, file_path)
    return file_path

## Model

### Get last posts

In [ ]:
def get_last_posts(df_notion, df_posts, owner, col_date="PUBLISHED_DATE", update_all=False):
    # Update all posts
    df_update = df_posts.copy()
    
    if update_all:
        return df_update

    # Filter to get owner posts in Notion DB
    if len(df_notion) > 0:
        df_owner = df_notion[df_notion.Author == owner]
        if len(df_owner) > 0:
            df_update = df_update[:LINKEDIN_POSTS_UPDATE]        
    return df_update

### Get posts engagements

In [ ]:
def get_engagements(interaction, post_url):
    df = pd.DataFrame()
    try:
        if interaction == "LIKES":
            df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(post_url)
        elif interaction == "COMMENTS":
            df = linkedin.connect(LI_AT, JSESSIONID).post.get_comments(post_url)
    except Exception as e:
        if e.response.status_code:
            print(e)
    return df

In [ ]:
def update_engagements(df_posts,
                        df_interaction,
                        interaction,
                        csv_output,
                        no_posts=LINKEDIN_POSTS_UPDATE,
                        min_updated_time=LINKEDIN_MIN_UPDATED_TIME):
    # Init
    df_out = df_interaction.copy()
    
    # Get all interactions if dataframe init empty or not complete
    if len(df_posts) == 0:
        return pd.DataFrame()

    if len(df_out) > 0:
        if "DATE_EXTRACT" in df_out.columns:
            last_update_date = df_out["DATE_EXTRACT"].max()
            time_last_update = datetime.now() - datetime.strptime(last_update_date, "%Y-%m-%d %H:%M:%S")
            minute_last_update = time_last_update.total_seconds() / 60
            if minute_last_update > min_updated_time:
                df_posts = df_posts[:no_posts]
            else:
                print(f"🛑 Nothing to update. Last update done {int(minute_last_update)} minutes ago.")
                return df_out.reset_index(drop=True)
    else:
        df_posts["SCENARIO"] = pd.to_datetime(df_posts["PUBLISHED_DATE"].str[:-6]).dt.strftime(PERIOD)
        df_posts = df_posts[df_posts["SCENARIO"] == datetime.now().strftime(PERIOD)].reset_index(drop=True)
        
    # Loop on posts
    for index, row in df_posts.iterrows():
        df_update = pd.DataFrame()
        post_title = row.TITLE
        post_author = row.AUTHOR_NAME
        post_url = row.POST_URL
        post_date = row.PUBLISHED_DATE
        count_interactions = row[interaction]
        print(f"🔄 {index+1} - Update started on: '{post_title}' ({post_url})")
        
        # Get interactions from post URL
        if len(df_interaction) > 0:
            tmp_df = df_interaction[df_interaction.POST_URL == post_url]
            no_interactions = len(tmp_df)
            if count_interactions != no_interactions:
                print(f"--> {count_interactions} post interaction count vs {no_interactions} interactions.")
                df_update = get_engagements(interaction, post_url)
            else:
                print("--->🛑 Nothing to update.")
        else:
            df_update = get_interactions(interaction, post_url)
        
        # Concat dataframe and save dataframe in CSV
        if len(df_update) > 0:
            print(f"---> {len(df_update)} interactions fetched.")
            df_update['TITLE'] = post_title
            df_update['AUTHOR_NAME'] = post_author
            df_update['PUBLISHED_DATE'] = post_date
            keys = ["POST_URL", "PROFILE_ID"]
            if interaction == "COMMENTS":
                keys = ["POST_URL", "PROFILE_ID", "CREATED_TIME"]
            df_out = pd.concat([df_update, df_out]).drop_duplicates(keys, keep="first")
            output_path = save_data(df_out, csv_output)
            
    # Add dependency in production
    print(f"✅ {len(df_out)} '{interaction}' fetched.")
    # Return all interactions
    return df_out.reset_index(drop=True)

In [ ]:
def create_engagements_db(df_likes, df_comments):
    # Init outputs
    df = pd.DataFrame()
    
    # Dataframe likes
    df_likes["REACTION"] = "LIKES"

    # Dataframe comments
    df_comments["REACTION"] = "COMMENTS"
    
    # Concat
    df = pd.concat([df_likes, df_comments]).fillna("Not defined").sort_values(by="PUBLISHED_DATE", ascending=False)
    
    # Cleaning
    to_keep = [
        "PROFILE_ID",
        "PROFILE_URL",
        "PUBLIC_ID",
        "FIRSTNAME",
        "LASTNAME",
        "FULLNAME",
        "OCCUPATION",
        "REACTION",
        "TEXT",
        "TITLE",
        "PUBLISHED_DATE",
        "AUTHOR_NAME",
        "POST_URL",
    ]
    df = df[to_keep]
    
    print(f"✅ {len(df)} interactions fetched.")
    return df.reset_index(drop=True)

### Get fans

In [ ]:
def get_fans(df):
    # Init
    df_fans = df.copy()
    
    # Filter
    df_fans["SCENARIO"] = pd.to_datetime(df_fans["PUBLISHED_DATE"].str[:-6]).dt.strftime(PERIOD)
    df_fans = df_fans[(df_fans["SCENARIO"] == datetime.now().strftime(PERIOD)) & (df_fans["FULLNAME"] != OWNER)].reset_index(drop=True)

    # Groupby
    to_group = ["PROFILE_URL", "FULLNAME", "OCCUPATION"]
    to_agg = {"PROFILE_ID": "count"}
    
    df_fans = df_fans.groupby(to_group, as_index=False).agg(to_agg)
    df_fans = df_fans.sort_values(by="PROFILE_ID", ascending=False).reset_index(drop=True)
    return df_fans

### Get fans data

In [ ]:
def get_fans_data(df):
    # Init
    df_fans = df.copy()
    
    # Filter
    df_fans = df_fans.drop_duplicates(["PROFILE_ID", "REACTION"], keep="last").reset_index(drop=True)
    df_fans["SCENARIO"] = pd.to_datetime(df_fans["PUBLISHED_DATE"].str[:-6]).dt.strftime(PERIOD)
    df_fans = df_fans[(df_fans["SCENARIO"] == datetime.now().strftime(PERIOD)) & (df_fans["FULLNAME"] != OWNER)].reset_index(drop=True)
    return df_fans

### Check fans

In [ ]:
def check_new_fans(df_fans, df_update_fans):
    # Concat data
    df_out = pd.concat([df_update_fans, df_fans]).drop_duplicates(["PROFILE_URL", "FULLNAME", "OCCUPATION"], keep=False)
    
    if len(df_out) == 0:
        return pd.DataFrame()
    else:
        return df_update_fans

### Transform UTC to local

In [ ]:
def utc_to_local(datetime_string,
                 datetime_format='%Y-%m-%d %H:%M:%S%z',
                 timezone=TIMEZONE):
    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    # utc = datetime.utcnow()
    utc = datetime.strptime(datetime_string, datetime_format)

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    local = utc.astimezone(to_zone)
    return local#.strftime(DATETIME_FORMAT)

### Get ranking

In [ ]:
def get_ranking(df_init,
                col_label,
                col_value,
                agg_value,
                label_max_len=100,
                ranking=10):
    # Init variable
    df = df_init.copy()
    
    # Groupby period
    df[col_value] = df[col_value].astype(float)
    df = df.groupby(col_label, as_index=False).agg({col_value: agg_value})
    
    # Rename column
    to_rename = {
        col_label: "LABEL",
        col_value: "VALUE"
    }
    df = df.rename(columns=to_rename)
    
    # Plotly: Label display
    df["LABEL_D"] = df.apply(lambda row: row["LABEL"].split("?")[0].split(".")[0], axis=1)
    df["LABEL_D"] = df.apply(lambda row: row["LABEL_D"] if len(row["LABEL_D"]) < 50 else row["LABEL_D"][:47] + "...", axis=1)
    
    # Plotly: Value display
    df["VALUE_D"] = "<b><span style='font-family: Arial;'>" + df["VALUE"].map("{:,.0f}".format).str.replace(",", " ") + "</span></b>"
    
    # Ranking
    df = df.sort_values(by="VALUE")
    return df[-ranking:].reset_index(drop=True)

### Get views reach

In [ ]:
def get_views_wdh(df_init,
                  col_date,
                  x_axis,
                  y_axis,
                  col_value,
                  type_value
                 ):
    # Init variable
    df = df_init.copy()
        
    # Setup date column and create X and Y axis analysis
    df[col_value] = df[col_value].astype(int)
    df[col_date] = pd.to_datetime(df[col_date].str[:18])
    df["X_AXIS"] = df[col_date].dt.strftime(x_axis)
    df["Y_AXIS"] = df[col_date].dt.strftime(y_axis)
    df = df.rename(columns={col_value: "VALUE"})
    
    # Filter data
    month_min = datetime.now() + relativedelta(months=-1)
    df = df[df[col_date].dt.strftime("%Y%m").astype(int) >= int(month_min.strftime(ORDER))]
    
    # Groupby
    to_group = [
        "X_AXIS",
        "Y_AXIS",
    ]
    df = df.groupby(to_group, as_index=False).agg({"VALUE": type_value})
    
    # Create empty value
    d = df["X_AXIS"].max()
    d2 = df["X_AXIS"].min()
    for x in range(int(d2), int(d)+1):
        data = [
            {"X_AXIS": x, "Y_AXIS": "1", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "2", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "3", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "4", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "5", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "6", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "7", "VALUE": 0},
        ]
        tmp_df = pd.DataFrame(data)
        df = pd.concat([tmp_df, df])
        
    
    # Group by with empty values
    df["X_AXIS"] = df["X_AXIS"].astype(int)
    df = df.groupby(to_group, as_index=False).agg({"VALUE": "sum"})
        
    # Sort values
    df = df.sort_values(by=["X_AXIS", "Y_AXIS"], ascending=[True, False])
    return df.reset_index(drop=True)

### Get posts frequency

In [ ]:
def get_frequency(df_init,
                  col_date,
                  x_axis,
                  y_axis,
                  col_value,
                  type_value
                 ):
    # Init variable
    df = df_init.copy()
    
    # Setup date column and create X and Y axis analysis
    df[col_date] = pd.to_datetime(df[col_date].str[:18])
    df["X_AXIS"] = df[col_date].dt.strftime(x_axis)
    df["Y_AXIS"] = df[col_date].dt.strftime(y_axis)
    df = df.rename(columns={col_value: "VALUE"})
    
    # Groupby
    to_group = [
        "X_AXIS",
        "Y_AXIS",
    ]
    df = df.groupby(to_group, as_index=False).agg({"VALUE": type_value})

    # Create empty value
    d = datetime.now() + MonthEnd(1)
    d2 = df["X_AXIS"].min()
    idx = pd.date_range(d2, d, freq="m", normalize=True)
    for x in idx:
        x = x.strftime("%Y-%m")
        data = [
            {"X_AXIS": x, "Y_AXIS": "1", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "2", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "3", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "4", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "5", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "6", "VALUE": 0},
            {"X_AXIS": x, "Y_AXIS": "7", "VALUE": 0},
        ]
        tmp_df = pd.DataFrame(data)
        df = pd.concat([tmp_df, df])
        
    # Group by with empty values
    df = df.groupby(to_group, as_index=False).agg({"VALUE": "sum"})
        
    # Sort values
    month_min = datetime.now() + relativedelta(months=-MONTH_ROLLING)
    df = df[df["X_AXIS"].str.replace("-", "").astype(int) >= int(month_min.strftime(ORDER))]
    df = df.sort_values(by=["X_AXIS", "Y_AXIS"], ascending=[True, False])
    return df.reset_index(drop=True)

### Get trend barchart

In [ ]:
def get_trend(df_init,
              col_date,
              col_value,
              agg_value,
              period_rolling=MONTH_ROLLING):
    # Init variable
    df = df_init.copy()
    
    # Groupby period
    if isinstance(col_value, list):
        df["VALUE"] = 0
        for c in col_value:
            df[c] = df[c].astype(float)
            df["VALUE"] = df["VALUE"] + df[c]    
        col_value = "VALUE"
    elif agg_value == "sum":
        df[col_value] = df[col_value].astype(float)
    df[col_date] = pd.to_datetime(df[col_date].str[:-6]).dt.strftime(DATE_FORMAT)
    df = df.groupby(col_date, as_index=False).agg({col_value: agg_value})
    
    # Rename column
    to_rename = {
        col_date: "DATE_ISO",
        col_value: "VALUE"
    }
    df = df.rename(columns=to_rename)
    
    # Reindex value
    d = datetime.now().date()
    d2 = df.loc[df.index[0], "DATE_ISO"]
    idx = pd.date_range(d2, d, freq = "D")    
    df.set_index("DATE_ISO", drop=True, inplace=True)
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    df["DATE_ISO"] = pd.DatetimeIndex(df.index)
    
    # Groupby month
    df["DATE"] = pd.to_datetime(df["DATE_ISO"], format=DATE_FORMAT).dt.strftime(PERIOD)
    # Plotly: Date display
    df["DATE_D"] = pd.to_datetime(df["DATE_ISO"], format=DATE_FORMAT).dt.strftime(PERIOD_D)
    df = df.groupby(["DATE", "DATE_D"], as_index=False).agg({"VALUE": "sum"})

    # Calc variation
    for index, row in df.iterrows():
        if index > 0:
            n = df.loc[df.index[index], "VALUE"]
            n_1 = df.loc[df.index[index-1], "VALUE"]
            df.loc[df.index[index], "VALUE_COMP"] = n_1
            df.loc[df.index[index], "VARV"] = n - n_1
            if n_1 > 0:
                df.loc[df.index[index], "VARP"] = (n - n_1) / abs(n_1)
    df = df.fillna(0)
    
    # Plotly: Value display
    df["VALUE_D"] = "<b><span style='font-family: Arial;'>" + df["VALUE"].map("{:,.0f}".format).str.replace(",", " ") + "</span></b>"
    
    # Plotly: Variation display
    df["VARV_D"] = df["VARV"].map("{:,.0f}".format).str.replace(",", " ")
    df.loc[df["VARV"] >= 0, "VARV_D"] = "+" + df["VARV_D"]
    df["VARP_D"] = df["VARP"].map("{:,.0%}".format).str.replace(",", " ")
    df.loc[df["VARP"] >= 0, "VARP_D"] = "+" + df["VARP_D"]
    
    # Plotly: hovertext
    df["TEXT"] = ("<b><span style='font-size: 14px;'>" + df["DATE_D"].astype(str) + ": " + df["VALUE_D"] + "</span></b><br>"
                  "<span style='font-size: 12px;'>" + f"{PERIOD_TEXT}: " + df["VARV_D"] + " (" + df["VARP_D"] + ")</span>")
    
    # Return month rolling
    return df[-period_rolling:].reset_index(drop=True)

### Get trend barline

In [ ]:
def get_trend_barline(df_init,
                      col_date,
                      col_value,
                      agg_value,
                      period_rolling=MONTH_ROLLING):
    # Init variable
    df = df_init.copy()
    
    # Groupby period
    df[col_date] = pd.to_datetime(df[col_date]).dt.strftime(PERIOD)
    df = df.groupby(col_date, as_index=False).agg({col_value: agg_value})
    
    # Rename column
    to_rename = {
        col_date: "DATE_ISO",
        col_value: "VARV"
    }
    df = df.rename(columns=to_rename)
    
    # Reindex value
    d = datetime.now().date()
    d2 = df.loc[df.index[0], "DATE_ISO"]
    idx = pd.date_range(d2, d, freq = "D")    
    df.set_index("DATE_ISO", drop=True, inplace=True)
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    df["DATE_ISO"] = pd.DatetimeIndex(df.index)
    
    # Groupby month
    df["DATE"] = pd.to_datetime(df["DATE_ISO"], format=DATE_FORMAT).dt.strftime(PERIOD)
    # Plotly: Date display
    df["DATE_D"] = pd.to_datetime(df["DATE_ISO"], format=DATE_FORMAT).dt.strftime(PERIOD_D)
    df = df.groupby(["DATE", "DATE_D"], as_index=False).agg({"VARV": "sum"})
    
    # Add value col
    df.insert(loc=2, column="VALUE", value=df["VARV"].cumsum())
    df.insert(loc=3, column="VALUE_COMP", value=df["VALUE"] - df["VARV"])
    df["VARP"] = df["VARV"] / df["VALUE_COMP"]
    
    # Plotly: Value display
    df["VALUE_D"] = "<b><span style=font-size: 16px; style=font-family: Arial>" + df["VALUE"].map("{:,.0f}".format).str.replace(",", " ") + "</span></b>"
    
    # Plotly: Variation display
    df["VARV_D"] = df["VARV"].map("{:,.0f}".format).str.replace(",", " ")
    df.loc[df["VARV"] >= 0, "VARV_D"] = "+" + df["VARV_D"]
    df["VARP_D"] = df["VARP"].map("{:,.0%}".format).str.replace(",", " ")
    df.loc[df["VARP"] >= 0, "VARP_D"] = "+" + df["VARP_D"]
    
    # Plotly: hovertext
    df["TEXT"] = ("<b><span style='font-size: 14px;'>" + df["DATE_D"].astype(str) + ": " + df["VALUE_D"] + "</span></b><br>"
                  "<span style='font-size: 12px;'>" + f"{PERIOD_TEXT}: " + df["VARV_D"] + " (" + df["VARP_D"] + ")</span>")
    
    # Return month rolling
    return df[-period_rolling:].reset_index(drop=True)

### Get trend linechart

In [ ]:
def get_trend_linechart(df_init,
                        col_date,
                        col_value,
                        agg_value,
                        period_rolling=MONTH_ROLLING):
    # Init variable
    df = df_init.copy()
    
    # Groupby period
    df["ENGAGEMENTS"] = df["LIKES"].astype(int) + df["COMMENTS"].astype(int)  + df["SHARES"].astype(int)
    df["VIEWS"] = df["VIEWS"].astype(int)
    df[col_date] = pd.to_datetime(df[col_date].str[:-6]).dt.strftime(PERIOD)
    df = df.groupby(col_date, as_index=False).agg({"ENGAGEMENTS": agg_value, "VIEWS": agg_value})
    
    # Rename column
    to_rename = {
        col_date: "DATE_ISO",
    }
    df = df.rename(columns=to_rename)
    
    # Reindex value
    d = datetime.now().date()
    d2 = df.loc[df.index[0], "DATE_ISO"]
    idx = pd.date_range(d2, d, freq = "D")    
    df.set_index("DATE_ISO", drop=True, inplace=True)
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    df["DATE_ISO"] = pd.DatetimeIndex(df.index)
    
    # Groupby month
    df["DATE"] = pd.to_datetime(df["DATE_ISO"], format=DATE_FORMAT).dt.strftime(PERIOD)
    # Plotly: Date display
    df["DATE_D"] = pd.to_datetime(df["DATE_ISO"], format=DATE_FORMAT).dt.strftime(PERIOD_D)
    df = df.groupby(["DATE", "DATE_D"], as_index=False).agg({"ENGAGEMENTS": agg_value, "VIEWS": agg_value})
    df["VALUE"] = df["ENGAGEMENTS"].astype(int) / df["VIEWS"].astype(int)

    # Calc variation
    for index, row in df.iterrows():
        if index > 0:
            n = df.loc[df.index[index], "VALUE"]
            n_1 = df.loc[df.index[index-1], "VALUE"]
            df.loc[df.index[index], "VALUE_COMP"] = n_1
            df.loc[df.index[index], "VARV"] = n - n_1
            if n_1 > 0:
                df.loc[df.index[index], "VARP"] = (n - n_1) / abs(n_1)
    df = df.fillna(0)
    
    # Plotly: Date display
    df["DATE_D"] = pd.to_datetime(df["DATE"], format=PERIOD).dt.strftime(PERIOD_D)
    
    # Plotly: Value display
    df["VALUE_D"] = "<b><span style=font-size: 16px; style=font-family: Arial>" + df["VALUE"].map("{:,.2%}".format).str.replace(",", " ") + "</span></b>"
    
    # Plotly: Variation display
    df["VARV_D"] = (df["VARV"] * 100).map("{:,.2f} pts".format).str.replace(",", " ")
    df.loc[df["VARV"] >= 0, "VARV_D"] = "+" + df["VARV_D"]
    df["VARP_D"] = df["VARP"].map("{:,.0%}".format).str.replace(",", " ")
    df.loc[df["VARP"] >= 0, "VARP_D"] = "+" + df["VARP_D"]
    
    # Plotly: hovertext
    df["TEXT"] = ("<b><span style='font-size: 14px;'>" + df["DATE_D"].astype(str) + ": " + df["VALUE_D"] + "</span></b><br>"
                  "<span style='font-size: 12px;'>" + f"{PERIOD_TEXT}: " + df["VARV_D"] + " (" + df["VARP_D"] + ")</span>")
    
    # Return month rolling
    return df[-period_rolling:].reset_index(drop=True)

### Get notion content db

In [ ]:
def get_notion_df(database_url):
    # Get pages
    database_id = database_url.split("/")[-1].split("?v=")[0]
    pages = notion.connect(NOTION_TOKEN).database.query(database_id, query={})
    if len(pages) == 0:
        return pd.DataFrame()
    
    # Rename columns
    columns = pages[0].df().Name.tolist()
    to_rename = {}
    for i, v in enumerate(columns):
        to_rename[i] = v
    
    # Create dataframe from pages
    df = pd.DataFrame()
    for page in pages:
        values = page.df().drop("Type", axis=1).to_dict().get("Value")
        tmp_df = pd.DataFrame([values]).rename(columns=to_rename)
        content_url = tmp_df.loc[0, "Content URL"]
        if content_url == "None":
            notion.connect(NOTION_TOKEN).blocks.delete(page.id)
        else:
            df = pd.concat([df, tmp_df])

    df = df.sort_values(by="Publication Date", ascending=False).reset_index(drop=True)
    print("✅ Notion content DB:", len(df))
    return df

### Create horizontal barchart

In [ ]:
def create_horizontal_barchart(df,
                               label="LABEL_D",
                               value="VALUE",
                               value_d="VALUE_D"):
    # Init
    fig = go.Figure()
    
    # Create fig
    fig.add_trace(
        go.Bar(
            y=df[label],
            x=df[value],
            text=df[value_d],
            textposition="outside",
            marker=dict(color=COLOR),
            orientation="h"
        )
    )
    # Add logo
    fig.add_layout_image(
        dict(
            source=LOGO,
            xref="paper",
            yref="paper",
            x=-0.28,
            y=1.035,
            sizex=0.15,
            sizey=0.15,
            xanchor="right",
            yanchor="bottom"
        )
    )
    fig.update_traces(showlegend=False)
    # Plotly: Create title
    total_value = "{:,.0f}".format(df[value].sum()).replace(",", " ")
    title = f"<b><span style='font-size: 20px;'>{TITLE}</span></b><br><span style='font-size: 18px;'>Total {KPI_TITLE.lower()}: {total_value}</span>"
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
        margin_r=10,
        margin_l=10,
    )
    fig.update_xaxes(showticklabels=False)
    fig.show()
    return fig

### Create barchart

In [ ]:
def create_barchart(df,
                    label="DATE_D",
                    value="VALUE",
                    value_d="VALUE_D",
                    text="TEXT"):
    # Init
    fig = go.Figure()
    
    # Create fig
    fig.add_trace(
        go.Bar(
            x=df[label],
            y=df[value],
            text=df[value_d],
            textposition="outside",
            hoverinfo="text",
            hovertext=df[text],
            marker=dict(color=COLOR)
        )
    )
    # Add logo
    fig.add_layout_image(
        dict(
            source=LOGO,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.045,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom"
        )
    )
    fig.update_traces(showlegend=False)
    # Plotly: Create title
    total_value = "{:,.0f}".format(df[value].sum()).replace(",", " ")
    varv = df.loc[df.index[-1], "VARV"]
    varp = df.loc[df.index[-1], "VARP"]
    varv_d = "{:,.0f}".format(varv).replace(",", " ")
    varp_d = "{:,.0%}".format(varp).replace(",", " ")
    if varv >= 0:
        varv_d = f"+{varv_d}"
        varp_d = f"+{varp_d}"
    title = f"<b><span style='font-size: 20px;'>{TITLE}</span></b><br><span style='font-size: 18px;'>Total: {total_value} | {PERIOD_TEXT}: {varv_d} ({varp_d})</span>"
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
#         autosize=True,
        width=1200,
        height=600,
        margin_pad=10,
    )
#     config = {'displayModeBar': False, "responsive": True}
#     fig.show(config=config)
    fig.show()
    return fig

### Create barlinechart

In [ ]:
def create_barlinechart(df,
                        label="DATE_D",
                        value="VALUE",
                        value_d="VALUE_D",
                        text="TEXT",
                        yaxes_left=None,
                        yaxes_right=None):
    # Init
    fig = go.Figure()
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(
            x=df[label],
            y=df["VARV"],
            textposition="outside",
            hoverinfo="text",
            hovertext=df[text],
            marker=dict(color=COLOR2)
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=df[label],
            y=df[value],
            mode="lines+markers+text",
            hoverinfo="text",
            text=df[value_d],
            textposition="top left",
            hovertext=df[text],
            line=dict(color=COLOR, width=4),
        ),
        secondary_y=True,
    )
    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxes_left,
        title_font=dict(family="Arial", size=14, color="black"),
        secondary_y=False
    )
    fig.update_yaxes(
        title_text=yaxes_right,
        title_font=dict(family="Arial", size=14, color="black"),
        secondary_y=True
    )
    # Add logo
    fig.add_layout_image(
        dict(
            source=LOGO,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.045,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom"
        )
    )
    fig.update_traces(showlegend=False)
    # Plotly: Create title
    total_value = "{:,.0f}".format(df.loc[df.index[-1], "VALUE"]).replace(",", " ")
    varv = df.loc[df.index[-1], "VARV"]
    varp = df.loc[df.index[-1], "VARP"]
    varv_d = "{:,.0f}".format(varv).replace(",", " ")
    varp_d = "{:,.0%}".format(varp).replace(",", " ")
    if varv >= 0:
        varv_d = f"+{varv_d}"
        varp_d = f"+{varp_d}"
    title = f"<b><span style='font-size: 20px;'>{TITLE}</span></b><br><span style='font-size: 18px;'>Total: {total_value} | {PERIOD_TEXT}: {varv_d} ({varp_d})</span>"
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
    )
    fig.show()
    return fig

### Create linechart

In [ ]:
def create_linechart(df,
                    label="DATE_D",
                    value="VALUE",
                    value_d="VALUE_D",
                    text="TEXT"):
    # Init
    fig = go.Figure()
    
    # Create fig
    fig.add_trace(
        go.Scatter(
            x=df[label],
            y=df[value],
            mode="lines+markers+text",
            text=df[value_d],
            textposition="top center",
            hoverinfo="text",
            hovertext=df[text],
            line=dict(color=COLOR, width=4),
        )
    )
    # Add logo
    fig.add_layout_image(
        dict(
            source=LOGO,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.045,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom"
        )
    )
    fig.update_traces(showlegend=False)
    # Plotly: Create title
    total_eng = df["ENGAGEMENTS"].sum()
    total_views = df["VIEWS"].sum()
    total_value = "{:,.2%}".format(total_eng / total_views).replace(",", " ")
    varv = df.loc[df.index[-1], "VARV"] * 100
    varp = df.loc[df.index[-1], "VARP"]
    varv_d = "{:,.2f} pts".format(varv).replace(",", " ")
    varp_d = "{:,.0%}".format(varp).replace(",", " ")
    if varv >= 0:
        varv_d = f"+{varv_d}"
        varp_d = f"+{varp_d}"
    title = f"<b><span style='font-size: 20px;'>{TITLE}</span></b><br><span style='font-size: 18px;'>Average: {total_value} | {PERIOD_TEXT}: {varv_d} ({varp_d})</span>"
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
        yaxis = dict(
            range=[0, df[value].max() * 1.1],
            tickmode='array',
            tickformat='.1%'
        )
    )
    fig.show()
    return fig

### Create heatmap

In [ ]:
def create_heatmap(df,
                   x_value="X_AXIS",
                   y_value="Y_AXIS",
                   z_value="VALUE",
                   x_format="%H",
                   x_format_d="%H",
                   text="views",
                   ):
    
    # Add display values
    df["X_AXIS_D"] = pd.to_datetime(df[x_value], format=x_format).dt.strftime(x_format_d)
    df["Y_AXIS_D"] = df.apply(lambda row: calendar.day_name[int(row[y_value]) - 1], axis=1)
    df["TEXT"] = df[z_value].astype(str) + f" {text} on " + df["Y_AXIS_D"] + "s, " + df["X_AXIS_D"]

    # Create graph data
    x = sorted(df[x_value].unique().tolist())
    y = sorted(df[y_value].unique().tolist(), reverse=True)
    def get_values(df, y, value):
        values = []
        for i in y:
            tmp = df[df[y_value] == i].reset_index(drop=True)
            data = tmp[value].tolist()
            values.append(data)
        return values
    z = get_values(df, y, z_value)
    hovertext = get_values(df, y, "TEXT")
    
    # Colors
    colors = [
        [0.00, "#e7f4fa"],
        [0.01, "#b7def1"],
        [0.25, "#88c9e8"],
        [0.50, "#59b3df"],
        [1.00, "#1293d2"]
    ]

    # Create fig
    fig = go.Figure(
        data=go.Heatmap(
            x=df["X_AXIS_D"].unique().tolist(),
            y=df["Y_AXIS_D"].unique().tolist(),
            z=z,
            text=hovertext,
            hoverinfo="text",
            type='heatmap',
            colorscale=colors,
            hoverongaps=False,
        )
    )
    fig.add_layout_image(
        dict(
            source=LOGO,
            xref="paper",
            yref="paper",
            x=-0.01,
            y=1.045,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom"
        )
    )
    fig.update_traces(xgap=10,
                      ygap=10,
                      selector=dict(type='heatmap'),
                      showscale=False)
    total_value = "{:,.0f}".format(df[z_value].sum()).replace(",", " ")
    fig.update_layout(
        title = f"<b><span style='font-size: 20px;'>{TITLE}</span></b><br><span style='font-size: 18px;'>Total {text}: {total_value}</span>",
        title_x=0.08,
        title_font=dict(family="Arial", size=20, color="black"),
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        yaxis_scaleanchor="x"
    )
    fig.show()
    return fig

### Calc variation

In [ ]:
def calc_value(df_init, value="VALUE", value_comp="VALUE_COMP", varv="VARV", varp="VARP"):
    v1 = 0
    v2 = 0
    v3 = 0
    v4 = 0
    df = df_init.reset_index(drop=True)
    if len(df) > 0:
        if value in df.columns:
            v1 = df.loc[df.index[-1], value]
        if value_comp in df.columns:
            v2 = df.loc[df.index[-1], value_comp]
        if varv in df.columns:
            v3 = df.loc[df.index[-1], varv]
        if varp in df.columns:
            v4 = df.loc[df.index[-1], varp]
        return v1, v2, v3, v4
    else:
        return 0, 0, 0, 0

### Run pipeline

In [ ]:
GIT_REPO_NAME = naas.secret.get("GIT_REPO_NAME")

def run_pipeline(pipeline, folder):
    for notebook in pipeline:
        print("➡️ Started running notebook:", notebook)
        if not path.exists(notebook):
            notebook = path.join(GIT_REPO_NAME, folder, notebook)
        %run "$notebook"
        print("\n")

## Output

### Save data

In [ ]:
def save_data(df, file_path, storage="Naas"):
    output_path = None
    if len(df) > 0:
        df["DATE_EXTRACT"] = datetime.now().strftime(DATETIME_FORMAT)
    try:
        if storage == "Naas":
            if not naas.is_production():
                output_path = path.join(DEV_DIR, file_path)
                df.to_csv(output_path, index=False)
                naas.dependency.add(output_path)
            else:
                output_path = path.join(PROD_DIR, file_path)
                df.to_csv(output_path, index=False)
            print("✅ Dataframe successfully saved in CSV:", file_path)
        elif storage == "AWS":
            output_path = file_path
            wr.s3.to_parquet(
                df=df.astype(str),
                path=file_path,
                dataset=True,
                mode="overwrite"
            )
            print("✅ Dataframe successfully saved in AWS:", file_path)
    except Exception as e:
        return e
#     return output_path

### Save asset

In [ ]:
def save_graph_asset(obj, file_path, image=False):
    # Get output
    if naas.is_production():
        output_path = path.join(PROD_DIR, file_path)
    else:
        output_path = path.join(DEV_DIR, file_path)
        
    # Save asset
    if file_path.endswith(".csv"):
        obj["DATE_EXTRACT"] = datetime.now().strftime(DATETIME_FORMAT)
        obj.to_csv(output_path, index=False)
    elif file_path.endswith(".html"):
        obj.write_html(output_path)
    elif file_path.endswith(".png") and not image:
        obj.write_image(output_path)
    elif file_path.endswith(".png") and image:
        obj.to_file(output_path)
    return output_path

### Get logo from platform

In [ ]:
def get_logo(platform, page_icon):
    # Manage page icon
    if page_icon is None:
        if platform == LINKEDIN:
            page_icon = LINKEDIN_LOGO
        elif platform == YOUTUBE:
            page_icon = YOUTUBE_LOGO
        elif platform == TWITTER:
            page_icon = TWITTER_LOGO
    return page_icon

### Delte page blocks

In [ ]:
def delete_page_blocks(page):  
    # Delete all blocks
    blocks = page.get_blocks()
    if len(blocks) > 0:
        while True:
            for block in blocks:
                notion.connect(NOTION_TOKEN).blocks.delete(block.id)
            blocks = page.get_blocks()
            if len(blocks) == 0:
                break
    page.update()
    return page

### Update "Report" Notion DB

In [ ]:
def update_report_status(report,
                         platform,
                         owner,
                         df=pd.DataFrame(),
                         image_link=None,
                         html_link=None,
                         csv_link=None,
                         page_icon=None,
                         ):
    # Create asset dataframe
    new_asset = {
        "REPORT": report,
        "PLATFORM": platform,
        "OWNER": owner,
        "IMAGE_LINK": image_link,
        "HTML_LINK": html_link,
        "CSV_LINK": csv_link,
        "DATE_EXTRACT": None,
    }
    # Get asset
    df_asset = get_data(ASSETS)
    if naas.is_production():
        tmp_asset = df_asset[(df_asset["REPORT"] == report) & (df_asset["PLATFORM"] == platform) & (df_asset["OWNER"] == owner)].reset_index(drop=True)
        if len(tmp_asset) > 0:
            if image_link:
                image_link = tmp_asset.loc[0 , "IMAGE_LINK"]
            if html_link:
                html_link = tmp_asset.loc[0 , "HTML_LINK"]
            if csv_link:
                csv_link = tmp_asset.loc[0 , "CSV_LINK"]

    # Save assets
    df_new = pd.DataFrame([new_asset])
    df_new = df_new.sort_values(by="DATE_EXTRACT").reset_index(drop=True)
    df_asset = pd.concat([df_asset, df_new]).drop_duplicates(["REPORT", "PLATFORM", "OWNER"], keep="last").reset_index(drop=True)
    save_data(df_asset, ASSETS)
    
    # Manage page icon
    page_icon = get_logo(platform, page_icon)
    
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})

        # Check if image already exists
        blocks = page.get_blocks()
        for block in blocks:
            content_block = getattr(block, block.type)
            if block.type == "image":
                image_url = block.image.external.url
                if image_url.startswith("https://public.naas.ai/"):
                    notion.connect(NOTION_TOKEN).blocks.delete(block.id)
            if block.type == "paragraph":
                if len(block.paragraph.text) > 0:
                    text = block.paragraph.text[0].text.content
                    if text == "Open dynamic chart":
                        notion.connect(NOTION_TOKEN).blocks.delete(block.id)
                    if text == "Download CSV":
                        notion.connect(NOTION_TOKEN).blocks.delete(block.id)
        if image_link:
            page.image(image_link)
        if html_link:
            res = page.paragraph("Open dynamic chart")
            res.paragraph.text[0].href = html_link
            res.paragraph.text[0].text.link = Link(html_link)
            res = page.rich_text("Chart", "Open dynamic chart")
            res.rich_text[0].href = html_link
            res.rich_text[0].text.link = Link(html_link)
        if csv_link:
            res = page.paragraph("Download CSV")
            res.paragraph.text[0].href = csv_link
            res.paragraph.text[0].text.link = Link(csv_link)
                
        # Update value
        value, value_comp, varv, varp = calc_value(df)
        not_stats = ["Frequency",
                     "Views reach",
                     "World cloud",
                     "Top posts",
                     "Top fans",
                     "New fans"]
        if report not in not_stats and platform != YOUTUBE:
            page.number("This month", float(value))
            page.number("Last month", float(value_comp))
            page.number("Variation", float(varv))
            page.number("Variation %", round(float(varp), 4))
            if varv > 0:
                sentiment = f"🟢 +{str(varv)} (+{str(round(varp * 100, 1))}%)"
            elif varv < 0:
                sentiment = f"🔴 {str(varv)} ({str(round(varp * 100, 1))}%)"
            elif varv == 0:
                sentiment = "🟠"
            page.rich_text("Sentiment", sentiment)
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))

        # Create page in Notion
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)

### List new connections

In [ ]:
def list_new_connections(report,
                         platform,
                         owner,
                         df=pd.DataFrame(),
                         image_link=None,
                         html_link=None,
                         csv_link=None,
                         page_icon=None,
                         ):
    # Date now number
    date_now_number = int(datetime.now().strftime("%Y%m%d"))
    
    # Manage page icon
    page_icon = get_logo(platform, page_icon)
    
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})
            
        # Delete page blocks
        page = delete_page_blocks(page)
#         blocks = page.get_blocks()
#         stop_delete = False
#         if len(blocks) > 0:
#             while True:
#                 for block in blocks:
#                     if block.type == "heading_3":
#                         text = block.heading_3.text
#                         if len(text) > 0:
#                             date = text[0].text.content
#                             date_number = datetime.strptime(date, "%Y-%m-%d").strftime("%Y%m%d")
#                             if int(date_number) < date_now_number:
#                                 stop_delete = True
#                                 break
#                     notion.connect(NOTION_TOKEN).blocks.delete(block.id)
#                 blocks = page.get_blocks()
#                 if len(blocks) == 0 or stop_delete:
#                     break
#         page.update()
        dates = df.DATE.unique()
        for d in dates:
#             tmp_df = df[(df.DATE == d) & (df.DATE_NUMBER.astype(int) > date_now_number)].reset_index(drop=True)
            tmp_df = df[(df.DATE == d)].reset_index(drop=True)
            if len(tmp_df) > 0:
                page.heading_3(d)
                for index, row in tmp_df.iterrows():
                    res = page.bulleted_list_item(f"{row.FIRSTNAME} {row.LASTNAME} - {row.OCCUPATION}")
                    res.bulleted_list_item.text[0].href = row.PROFILE_URL
                    res.bulleted_list_item.text[0].text.link = Link(row.PROFILE_URL)

        # Create page in Notion
        value, value_comp, varv, varp = calc_value(csv_input)
        page.number("This month", float(value))
        page.number("Last month", float(value_comp))
        page.number("Variation", float(varv))
        page.number("Variation %", round(float(varp), 4))
        if varv > 0:
            sentiment = f"🟢 +{str(int(varv))} (+{str(round(varp * 100, 1))}%)"
        elif varv < 0:
            sentiment = f"🔴 {str(int(varv))} ({str(round(varp * 100, 1))}%)"
        elif varv == 0:
            sentiment = "🟠"
        page.rich_text("Sentiment", sentiment)
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)
    return page

### List top posts

In [ ]:
def list_top_posts(report,
                   platform,
                   owner,
                   df=pd.DataFrame(),
                   image_link=None,
                   html_link=None,
                   csv_link=None,
                   page_icon=None,
                  ):
    # Manage page icon
    page_icon = get_logo(platform, page_icon)
    
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})
            
        # Delete page blocks
        page = delete_page_blocks(page)
        
        kpis = ["ENGAGEMENT_SCORE", "VIEWS", "COMMENTS", "LIKES"]
        for kpi in kpis:
            heading_3 = f"By {kpi.lower()}"
            if kpi == "ENGAGEMENT_SCORE":
                heading_3 = "By engagement rate"
            page.heading_3(heading_3)
            tmp_df = df.sort_values(by=kpi, ascending=False).reset_index(drop=True)[:3]
            for index, row in tmp_df.iterrows():
                value = row[kpi]
                kpi_d = kpi.lower()
                title = f"{row.TITLE} ({value} {kpi_d})"
                if kpi == "ENGAGEMENT_SCORE":
                    value = "{:,.2%}".format(value)
                    title = f"{row.TITLE} ({value})"
                res = page.numbered_list_item(title)
                res.numbered_list_item.text[0].href = row.POST_URL
                res.numbered_list_item.text[0].text.link = Link(row.POST_URL)
            
        # Create page in Notion
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)

### List top fans

In [ ]:
def list_top_fans(report,
                  platform,
                  owner,
                  df=pd.DataFrame(),
                  image_link=None,
                  html_link=None,
                  csv_link=None,
                  page_icon=None,
                  ):
    # Manage dataframe empty
    if len(df) == 0:
        print(f"🛑 Nothing to update in Notion.")
        return
    else:
        df = df[:NO_TOP_FANS]
    
    # Manage page icon
    page_icon = get_logo(platform, page_icon)
    
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})
            
        # Delete page blocks
        page = delete_page_blocks(page)
        
        page.heading_3("Ranking this month")
        for index, row in df.iterrows():
            text = f"{row.FULLNAME} - {row.OCCUPATION} ({str(row.PROFILE_ID)})"
            res = page.numbered_list_item(text)
            res.numbered_list_item.text[0].href = row.PROFILE_URL
            res.numbered_list_item.text[0].text.link = Link(row.PROFILE_URL)

        # Create page in Notion
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)

### List new fans

In [ ]:
def list_new_fans(report,
                  platform,
                  owner,
                  df_engagements,
                  df,
                  page_icon=None,
                  ):
    # Manage dataframe empty
    if len(df_engagements) == 0:
        print(f"🛑 Nothing to update in Notion.")
        return
    
    # Manage page icon
    page_icon = get_logo(platform, page_icon)
    
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})
            
        # Create posts dataframe
        df_posts = df_engagements[["TITLE", "POST_URL", "PUBLISHED_DATE"]].drop_duplicates(["POST_URL"]).reset_index(drop=True)
        df_posts["SCENARIO"] = pd.to_datetime(df_posts["PUBLISHED_DATE"].str[:-6]).dt.strftime(PERIOD)
        df_posts = df_posts[(df_posts["SCENARIO"] == datetime.now().strftime(PERIOD))][:LINKEDIN_POSTS_UPDATE].reset_index(drop=True)
        
        # Delete page blocks
        page = delete_page_blocks(page)
#         stop_delete = False
#         if len(blocks) > 0:
#             from datetime import timedelta
#             df_posts["PUBLISHED_DATE"] = pd.to_datetime(df_posts["PUBLISHED_DATE"].str[:-6])
#             df_posts = df_posts[df_posts["PUBLISHED_DATE"] > (datetime.now() - timedelta(days=7))]
#             titles = df_posts["TITLE"].unique()
#             while True:
#                 for block in blocks:
#                     if block.type == "heading_3":
#                         text = block.heading_3.text
#                         if len(text) > 0:
#                             title = text[0].text.content
#                             if title not in titles:
#                                 stop_delete = True
#                                 break
#                     notion.connect(NOTION_TOKEN).blocks.delete(block.id)
#                 blocks = page.get_blocks()
#                 if len(blocks) == 0 or stop_delete:
#                     break
#             page.update()
    
        # Add new fans by posts
        for index, row in df_posts.iterrows():
            # Add posts
            res = page.heading_3(row.TITLE)
            res.heading_3.text[0].href = row.POST_URL
            res.heading_3.text[0].text.link = Link(row.POST_URL)
            
            # Add new
            tmp_df = df[df.POST_URL == row.POST_URL].reset_index(drop=True)
            def add_reactions(df, reaction):
                df = df[df.REACTION == reaction]
                if reaction == "LIKES":
                    page.paragraph(f"New profile who liked ({len(df)}):")
                elif reaction == "COMMENTS":
                    page.paragraph(f"New profile who commented ({len(df)}):")
                for index, row in df.iterrows():
                    text = f"{row.FULLNAME} - {row.OCCUPATION}"
                    res = page.bulleted_list_item(text)
                    res.bulleted_list_item.text[0].href = row.PROFILE_URL
                    res.bulleted_list_item.text[0].text.link = Link(row.PROFILE_URL)
                page.paragraph("")
            add_reactions(tmp_df, "COMMENTS")
            add_reactions(tmp_df, "LIKES")


        # Create page in Notion
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)

### Update Notion Content DB

In [ ]:
def update_dynamic_properties(page, row):
    # Page properties : dynamic
    page.number("Engagment score", round(float(row.ENGAGEMENT_SCORE), 4))
    page.number("Views", int(row.VIEWS))
    page.number("Likes", int(row.LIKES))
    page.number("Comments", int(row.COMMENTS))
    page.number("Shares", int(row.SHARES))
    return page

In [ ]:
def update_content_notion(df, database_url):
    # Decode database id
    database_id = database_url.split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id, query={})
    
    # Manage dataframe empty
    if len(df) == 0:
        print(f"🛑 Nothing to update in Notion.")
        return
    
    # Loop in data
    fillna = {
        "AUTHOR_URL": "",
        "TITLE": "",
        "TEXT": "",
        "COMPANY_MENTION": "",
        "PROFILE_MENTION": "",
        "TAGS": "",
        "TAGS_COUNT": 0,
        "LINKS": "",
        "LINKS_COUNT": 0,
        "EMOJIS": "",
        "EMOJIS_COUNT": 0,
        "CONTENT_TITLE": "",
        "CONTENT_URL": "",
        "CONTENT_ID": "",
        "POLL_ID": "",
        "POLL_QUESTION": "",
        "POLL_RESULTS": "",
        "IMAGE_URL": ""
    }
    df = df.fillna(fillna)
    for i, row in df.iterrows():
        title = row.TITLE
        content_title = row.CONTENT_TITLE
        if title is None and content_title is not None:
            title = f"Repost - {content_title}"
        elif title is None and content_title is None:
            title = "Repost"
        post_url = row.POST_URL
        print(post_url)
        
        # Create or update page
        page_new = True
        for page in pages:
            page_temp = page.df()
            page_id = page_temp.loc[page_temp.Name == "Content URL", "Value"].values
            if page_id == post_url:
                page_new = False
                break
        try:
            if page_new:
                # Create new page in notion
                page = notion.Page.new(database_id=database_id).create()

                # Page properties : static
                page.date("Publication Date", row.PUBLISHED_DATE)
                page.title("Name", title)
                page.select("Content type", row.CONTENT)
                page.select("Platform", "LinkedIn")
                page.select("Status", "Published ✨")
                page.select("Author", row.AUTHOR_NAME)
                profile_mention = row.PROFILE_MENTION
                if profile_mention is not None:
                    if len(profile_mention) > 2:
                        page.rich_text("Profile mention", profile_mention)
                company_mention = row.COMPANY_MENTION
                if company_mention is not None:
                    if len(company_mention) > 2:
                        page.rich_text("Company mention", company_mention)
                page.number("Nb tags", int(row.TAGS_COUNT))
                tags = row.TAGS
                if tags is None:
                    tags = ""
                else:
                    if len(tags) < 2:
                        tags = ""
                page.rich_text("Tags", tags)
                page.number("Nb emojis", int(row.EMOJIS_COUNT))
                emojis = row.EMOJIS
                if emojis is None:
                    emojis = ""
                else:
                    if len(emojis) < 2:
                        emojis = ""
                page.rich_text("Emojis", emojis)
                page.number("Nb links", int(row.LINKS_COUNT))
                links = row.LINKS
                if links is not None:
                    if len(links) > 2:
                        page.link("Links", links)
                page.number("Nb characters", int(row.CHARACTER_COUNT))
                page.link("Content URL", post_url)
                
                # Page blocks text
                page.heading_1("Text")
                text = row.TEXT
                if text is not None:
                    split_text = text.split("\n")
                    for t in split_text:
                        page.paragraph(t)
                    
                # Page blocks content
                image_url = row.IMAGE_URL
                content_url = row.CONTENT_URL
                poll_question = row.POLL_QUESTION
                if image_url or content_title or content_url or poll_question:
                    page.heading_1("Content")
                
                # Add image in content section
                if image_url:
                    page.heading_2("Image")
                    page.paragraph(image_url)
                    
                # Add post in content section
                if content_title:
                    page.heading_2("Media")
                    page.heading_3(content_title)
                
                if content_url:
                    page.paragraph(content_url)
                
                # Add poll graph in content section
                if poll_question:
                    page.heading_3("Poll")
                    page.paragraph(row.POLL_RESULTS)
                
                # Page properties : dynamic
                page = update_dynamic_properties(page, row)
                
                # Create page in Notion
                page.update()
                print(f"✅ Page '{title}' created in Notion.", '\n')
            else:
                # Page properties : dynamic
                page = update_dynamic_properties(page, row)
                
                # Update page
                page.update()
                print(f"📈 Post stats updated in notion for page '{title}'.", '\n')
        except Exception as e:
            print(f"❌ Error creating page '{title}' in Notion", e)
            print(row)
            raise(e)